### Tworzenie klasy "choir"

Klasa posiada metody:

***learn_old_songs(artists, pages)***

Parametry oznaczają listę artystów i liczbę stron, na któych ma się uczyć. Na jednej stronie znajduje się 100 piosenek.
Domyślnie uczy się na 100 piosenkach Queen.

***learn_to_sing(n)***

n to parametr uczenia modelu, od którego zależyjego złożoność i liczba słów, od której będziemy zaczynać nowe piosenki
Domyślnie n=2.

***sing_song(song_start, steps)***

Song_start to string zawierający tyle słów ile wynosi n. Steps to liczba kroków, w których piosenka będzie rozwijana.
Domyślnie liczba kroków to 30.
Jeśli nie podamy początku piosenki, chór wybierze losowy początek.

In [59]:
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib.request
import random
from typing import Dict

artists=['Paul Mccartney','Rod Stewart','Johnny Cash','Ray Charles']

class choir():
    
    def __init__(self):
        self.url = 'http://lyricsera.com/{art}-p{page}-lyrics.html'
 
       
    def learn_old_songs(self, artists: list=['queen'], pages: int = 2)  -> pd.DataFrame:
        print("Analyzing old songs...")
        songs = dict(title=[], lyrics=[])

        for artist in artists:
            print(artist)
            
            for page in range(1,pages+1):
                print("Page: {page}".format(page=page))
                url = 'http://lyricsera.com/{art}-p{page}-lyrics.html'.format(art = artist.lower().replace(" ","-"),page=page)
                request = urllib.request.urlopen(url)
                soup = BeautifulSoup(request, 'lxml')
                for link in self.extract_links(soup):
                    request_link = urllib.request.urlopen(link.get('href'))
                    soup_link = BeautifulSoup(request_link.read(), 'lxml')
                    song_text = self.preprocess_lyrics(str(soup_link.tt).lower())
                    songs['title'].append(link.get('title'))
                    songs['lyrics'].append(song_text)
        
        self.songs_db = pd.DataFrame.from_dict(songs)                    
    
    
    @staticmethod    
    def extract_links(soup):
            links = []
            for link in soup.find_all('a'):
                try:
                    if re.search(r"/[0-9]",link.get('href')):
                        links.append(link)
                except Exception as e:
                    print(e.args + ('failed for {link}'.format(link=link),))
            return links

    @staticmethod
    def preprocess_lyrics(lyrics):
        try:
            song_text = re.sub(r"[\[=,.!?()\"]", " ", lyrics)
            song_text = re.sub(r"<[a-z/]+>", " ", song_text)
            song_text = re.sub(r"\s+", " ", song_text)
            song_text = song_text.replace("chorus", " ").replace(" ' "," ")
            return song_text
        except Exception as e:
            print(e.args + ('preprocessing failed.',))
            return lyrics    

        
    def learn_to_sing(self, n: int = 2):
        
        if hasattr(self, 'songs_db') == False:
            print('Need to learn old songs first...')
            self.learn_old_songs()
        
        print("Learning how to sing...")
        self.n = n
        count = 1.0
        words, words_model = list(), dict()
        for lyrics in self.songs_db['lyrics']:
            words.extend(lyrics.split())
            for i in range(len(words)-n):
                song_part = ' '.join(words[i:i+n])
                if song_part not in words_model.keys():
                    words_model[song_part]=[]
                words_model[song_part].append(words[i+n])
                
            if round(10*count/len(self.songs_db['lyrics']))>round(10*(count-1)/len(self.songs_db['lyrics'])):
                print(round(100*count/len(self.songs_db['lyrics'])), '% done')
            count += 1
        
        self.model = words_model

        
    def sing_song(self, song_start: str = '', steps: int = 30):
       
        if hasattr(self, 'model') == False:
            print("Have to learn how to sing first")
            self.learn_to_sing()
        
        if song_start=='':
            r = random.randrange(len(self.model.keys()))
            song_start = list(self.model.keys())[r]
            
        if song_start in self.model.keys():
            song = list()
            words = song_start
            song.append(words)
            for _ in range(steps):
                possibilities = self.model[words]
                next_item = possibilities[random.randrange(len(possibilities))]
                song.append(next_item)
                words = ' '.join(song[-self.n:])
                words = ' '.join(words.split()[-self.n:])
            return ' '.join(song)
        else: print("Cannot start with those words. Please try a different beginning.")

In [57]:
m1 = choir()

In [ ]:
m1.sing_song()

In [60]:
m2 = choir()

In [61]:
artists = ['Paul Mccartney','Rod Stewart','Johnny Cash','Ray Charles']
m2.learn_old_songs(artists, 3)

Analyzing old songs...
Paul Mccartney
Page: 1
Page: 2
Rod Stewart
Page: 1
Page: 2
Johnny Cash
Page: 1
Page: 2
Ray Charles
Page: 1
Page: 2


In [62]:
m2.learn_to_sing(3)

Learning how to sing...
5 % done
15 % done
25 % done
35 % done
45 % done
55 % done
65 % done
75 % done
85 % done
95 % done


In [63]:
m2.sing_song()

"old massa day after day hearts torn in every way don't time fly when you're loving and laughing i've said goodbye so many times before ain't superstitious a black cat crossed my trail"

In [65]:
m2.sing_song(steps=40)

'you please listen to me we had a short talk and a long walk through the valley of the shadow of death i will fear no evil for you are with me ill be your salvation though the storm surrounding there are no'

In [66]:
m2.learn_to_sing(2)

Learning how to sing...
5 % done
15 % done
25 % done
35 % done
45 % done
55 % done
65 % done
75 % done
85 % done
95 % done


In [67]:
m2.sing_song()

"hand what a time there was a beautiful thing as long as we've been calling it love but when i looked into your beautiful eyes still no reflection did i begin to"

In [68]:
m2.sing_song("i did")

"i did i said a shoulder to shoulder all right one more celestial dance if only but it's cold outside i simply must go along the highwayhoney i want to come home"

In [69]:
m2.sing_song("you are", 50)

"you are alone with all your regrets you know don't you cry gonna be just fine mm-mm well i grew up quick and i won't let nobody hurt you no more and don't you meet today tomorrow forget you're the best friends i've found a more down hero if you'da taken an"